In [ ]:
import numpy as np

import scipy.ndimage as ndi

from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve
from astropy.nddata import CCDData
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy.nddata import Cutout2D
from astropy.modeling import models 

from photutils.isophote import EllipseGeometry, Ellipse

from petrofit.segmentation import make_catalog, plot_segments, get_source_position, get_source_ellip, get_source_elong, get_source_theta
from petrofit.photometry import make_radius_list, source_photometry, order_cat
from petrofit.utils import plot_target
from petrofit.petrosian import Petrosian, PetrosianCorrection
from petrofit.segmentation import masked_segm_image
from petrofit.models import sersic_enclosed_inv, sersic_enclosed, petrosian_profile
from petrofit.models import PSFModel
from petrofit.fitting import fit_model, model_to_image
import photutils

import time

import statmorph

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("..plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

# Load GalFit Generated Images 

We run this notebook once (restated) for every `galfit_simulated_sersic_n*.fits` file manually. Paramteres, except for `input_file`, are not changed during each run. 

In [ ]:
input_file = 'galfit_simulated_sersic_n0.5_psf.fits'

In [ ]:
fitting_image = CCDData.read(input_file, unit='electron / s')

In [ ]:
psf = None 

if True:
    # Load PSF image (2D array)
    psf = fits.getdata('f105_moffat_psf.fits.gz')
    #psf = fits.getdata('../../../galfit/galfit-example/EXAMPLE/robel_test/psf.fits')

    # Normalize PSF 
    psf = psf / psf.sum()

    # Note that the PSF shape is odd on all sides
    print("PSF Shape = {}".format(psf.shape))

    # Plot PSF and use vmax and vmin to show difraction spikes
    plt.title('PSF')
    plt.imshow(psf, vmin=0, vmax=psf.std()/10)
    plt.show()

In [ ]:
vmax = fitting_image.data.std()*3 # Use the image std as max and min of all plots 
vmin = - vmax 

In [ ]:
sigma_clipped_stats(fitting_image.data)

In [ ]:
#fitting_image.data += np.random.normal(loc=0, scale=7.746282e-07, size=fitting_image.data.shape)

In [ ]:
# Plot cutout that will be fit 
plt.imshow(fitting_image.data, vmin=vmin, vmax=vmax)
plt.title("Galaxy")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

In [ ]:
#fitting_image.header['BUNIT']

In [ ]:
#fitting_image.header['EXPTIME'] # seconds

In [ ]:
gain = 1000.00#fitting_image.header['EXPTIME']
gain 

# Segmentation 

In [ ]:
from petrofit.segmentation import make_catalog, plot_segments
from astropy.stats import sigma_clipped_stats

image_mean, image_median, image_stddev = sigma_clipped_stats(fitting_image.data, sigma=3)

threshold = image_stddev * 2

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 2**2


cat, segm, segm_deblend = make_catalog(    
    fitting_image.data, 
    threshold=threshold, 
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(cat))

In [ ]:
# Sort and select object of interest in the catalog
sorted_idx_list = order_cat(cat, key='area', reverse=True)
idx = sorted_idx_list[0] # index 0 is largest 
source = cat[idx]  # get source from the catalog 

plot_target(
    position=(source.maxval_xindex, source.maxval_yindex), 
    image=fitting_image.data, 
    size=200, 
    vmax=vmax*10, 
    vmin=vmin
)


In [ ]:
source.xcentroid, source.ycentroid

# StatMorph 

In [ ]:
start = time.time()
source_morphs = statmorph.source_morphology(
    fitting_image.data, segm_deblend, gain=gain, psf=psf)

stat_morph_time = time.time() - start
print('Time: %g s.' % (stat_morph_time))

In [ ]:
morph = source_morphs[idx]

In [ ]:
print('rpetro_ellip =', morph.rpetro_ellip)
print('rhalf_ellip =', morph.rhalf_ellip)
print('r20 =', morph.r20)
print('r80 =', morph.r80)
print('sersic_amplitude =', morph.sersic_amplitude)
print('sersic_rhalf =', morph.sersic_rhalf)
print('sersic_n =', morph.sersic_n)
print('sersic_xc =', morph.sersic_xc)
print('sersic_yc =', morph.sersic_yc)
print('sersic_ellip =', morph.sersic_ellip)
print('sersic_theta =', morph.sersic_theta)

# PetroFit

In [ ]:
def petrofit_source_morphology(source):
    max_pix = max(fitting_image.shape)//2

    r_list = make_radius_list(
        max_pix=max_pix, # Max pixel to go up to
        n=max_pix#min(max_pix, 1000) # the number of radii to produce 
    )

    # Photomerty 
    flux_arr, area_arr, error_arr = source_photometry(

        # Inputs 
        source, # Source (`photutils.segmentation.catalog.SourceCatalog`)
        fitting_image.data, # Image as 2D array 
        segm_deblend, # Deblended segmentation map of image
        r_list, # list of aperture radii

        # Options 
        cutout_size=max(r_list)*2, # Cutout out size, set to double the max radius  
        plot=False, 
    )

    p = Petrosian(r_list, area_arr, flux_arr)

    pc = PetrosianCorrection("./galfit_f105_moffat_psf_correction_gid.yaml")

    corrected_epsilon = pc.estimate_epsilon(
        p.r_half_light,
        p.concentration_index()[-1]

    )

    corrected_p = Petrosian(r_list, area_arr, flux_arr, uncertainties=error_arr,
                            epsilon=corrected_epsilon)


    image_x_0, image_y_0 = get_source_position(source)
    x_0, y_0 = image_x_0, image_y_0
    ellip = get_source_ellip(source)
    elong = get_source_elong(source)
    theta = get_source_theta(source)
    n = pc.estimate_n(
        p.r_half_light,
        p.concentration_index()[-1]
    )
    r_eff = corrected_p.r_half_light
    
    amplitude = 0 
    rhalf_ellip = 0
    rpetro_ellip = 0 
    
    ellip_failed = False
    try:
        # Define EllipseGeometry using ellip and theta
        g = EllipseGeometry(image_x_0, image_y_0, 1., ellip, theta)

        # Create Ellipse model 
        ellipse = Ellipse(fitting_image.data, geometry=g)

        # Fit isophote at r_eff
        iso = ellipse.fit_isophote(r_eff)
        
        # Get flux at r_eff
        amplitude = iso.intens
        rhalf_ellip = iso.eps
        
        # Fit isophote at r_eff
        iso_petro = ellipse.fit_isophote(corrected_p.r_petrosian)
        rpetro_ellip = iso_petro.eps

    except:
        print('could n|ot calculate EllipseGeometry for source', source)
        ellip_failed = True
        
    if ellip_failed or np.isnan(corrected_p.r_total_flux):
        return corrected_p, p, image_x_0, image_y_0, ellip, elong, theta, n, r_eff, amplitude,rhalf_ellip,rpetro_ellip, None
        
        

    model = None

    
    
    center_slack = 4

    sersic_model = models.Sersic2D(

            amplitude=amplitude,
            r_eff=r_eff,
            n=n,
            x_0=float(x_0),
            y_0=float(y_0),
            ellip=ellip, 
            theta=theta,

            bounds = {
                'amplitude': (1e-9, None),
                'r_eff': (0, None),
                'n': (0, 6),
                'ellip': (0, 1),
                'theta': (-2*np.pi, 2*np.pi),
                'x_0': (x_0 - center_slack/2, x_0 + center_slack/2),
                'y_0': (y_0 - center_slack/2, y_0 + center_slack/2),
            },
    )
    
    oversample = None
    if n > 3:
    
        oversample = ('x_0', 'y_0', 50, 10)

    print(repr(sersic_model))

    psf_sersic_model = PSFModel.wrap(sersic_model, psf=psf, oversample=oversample)
    psf_sersic_model.fixed['psf_pa'] = True
    
    print(psf_sersic_model)
    masked_image = masked_segm_image(source, fitting_image.data, segm_deblend, fill=np.nan, mask_background=True)
    print('fitting')

    fitted_model, fit_info = fit_model(
       fitting_image.data, psf_sersic_model,
        maxiter=100000,
        epsilon=1.4901161193847656e-08,
        acc=1e-09,
    )
    

    return corrected_p, p, image_x_0, image_y_0, ellip, elong, theta, n, r_eff, amplitude,rhalf_ellip,rpetro_ellip, fitted_model
    

In [ ]:
start = time.time()
petro_morph = petrofit_source_morphology(source)
cp, p, image_x_0, image_y_0, ellip, elong, theta, n, r_eff, amplitude,rhalf_ellip,rpetro_ellip, model = petro_morph 
petro_time = time.time() - start
print('Time: %g s.' % (petro_time))

# GalFit Params

In [ ]:
galfit_r_eff=15 
galfit_n= float(input_file.split('_')[3].replace('n', ''))
galfit_x_0=450
galfit_y_0=450
galfit_ellip=0
galfit_theta=0
galfit_total_mag = 26.563
galfit_total_flux = 10**((galfit_total_mag - 26.563)/(2.5))

galfit_amplitude = galfit_total_flux / sersic_enclosed(np.inf, 1., galfit_r_eff, galfit_n, ellip=galfit_ellip)


sersic_model = models.Sersic2D(amplitude=galfit_amplitude, 
                               r_eff=galfit_r_eff, 
                               n=galfit_n, 
                               x_0=galfit_x_0, 
                               y_0=galfit_y_0, 
                               ellip=galfit_ellip, 
                               theta=galfit_theta)



In [ ]:
sampler_r = np.arange(1, 100, 0.01)
model_petro = petrosian_profile(sampler_r, r_eff=sersic_model.r_eff, n=sersic_model.n)

sersic_petro = sampler_r[abs(0.2 - model_petro).argmin()]

In [ ]:
f99 = sersic_enclosed(np.inf, sersic_model.amplitude, sersic_model.r_eff, sersic_model.n, ellip=sersic_model.ellip)  * 0.99
r20 = sersic_enclosed_inv(f99*0.2/0.99, sersic_model.amplitude, sersic_model.r_eff, sersic_model.n, ellip=sersic_model.ellip)
r80 = sersic_enclosed_inv(f99*0.8/0.99, sersic_model.amplitude, sersic_model.r_eff, sersic_model.n, ellip=sersic_model.ellip )
f99, r20,  r80, 5*np.log10(r80/r20)

In [ ]:
sersic_enclosed_inv(f99, sersic_model.amplitude, sersic_model.r_eff, sersic_model.n, ellip=sersic_model.ellip)


In [ ]:
rows = ['amplitude', '$r_{eff}$', 'n', '$x_0$', '$y_0$', 'ellip', 'theta', '$r_{petro}$', '$r_{20}$', '$r_{80}$', '$C_{2080}$', 'Total Flux', 'Time']

gf_inputs = [sersic_model.amplitude.value, sersic_model.r_eff.value, sersic_model.n.value, sersic_model.x_0.value, sersic_model.y_0.value, sersic_model.ellip.value, sersic_model.theta.value, sersic_petro, r20,  r80, 5*np.log10(r80/r20), f99, 0]
sm_results = [morph.sersic_amplitude,  morph.sersic_rhalf,  morph.sersic_n,  morph.sersic_xc, morph.sersic_yc,  morph.sersic_ellip, morph.sersic_theta, morph.rpetro_ellip, morph.r20, morph.r80, morph.concentration, morph.flux_ellip, stat_morph_time]
pf_results = [model.amplitude.value, model.r_eff.value, model.n.value, model.x_0.value, model.y_0.value, model.ellip.value, model.theta.value, cp.r_petrosian, cp.fraction_flux_to_r(0.2), cp.fraction_flux_to_r(0.8), cp.c2080, cp.total_flux, petro_time]

gf_inputs = [float('{:0.4e}'.format(i)) for i in gf_inputs]
sm_results = [float('{:0.4e}'.format(i)) for i in sm_results]
pf_results = [float('{:0.4e}'.format(i)) for i in pf_results]

t = Table(data=[rows, gf_inputs, pf_results, sm_results], names=['Values', 'GALFIT Inputs', 'PetroFit', 'statmorph'])
t

In [ ]:
t.write(input_file.replace('.fits', '_comparison.csv'), overwrite=True)